In [146]:
import numpy as np  # numpy is a numerical library for Python
import itertools
from plotly import express as ex
import pandas as pd
import ipywidgets as widgets
from ipywidgets import interact, fixed

# MJ2383: Lab 1 - Computing Levelised Cost of Electricity

In this lab, we are going to explore computation of an important metric called **levelised cost of electricity**, or LCOE for short.

The lab is broken up into several stages each of which culminates in writing a piece of Python code or doing some analysis. Don't worry if you've never written any Python before or done any programming - it is actually as easy as (or easier than) writing down equations.

In [stage 1](#Stage-1---Compute-LCOE), we'll write a Python function which computes the levelised cost of electricity from input data. We'll then test that this function produces the result we expect. You'll be able to check that your function works at the end of the stage.

In [stage 2](#Stage-2---Explore-Parameter-Values), we'll then use the function you wrote to compute LCOE for one technology and explore what happens when you change the parameter values.

in [stage 3](#Stage-3:-Comparing-technologies), we'll use the LCOE function again to compare technologies against one another, and develop screen curves that allow us to visualise the trade offs between technologies as a function of load-factor.

## Housekeeping

- This is a Jupyter notebook. A notebook contains cells. Each cell contains either descriptive text (like this) or Python code.  You execute a cell by pressing `Shift + Enter`.  Or, you can use the `Run` button in the menu bar at the top of the page.

- Don't worry, you cannot break anything. If you think you've made a mistake, you can delete the content and try again. If you get really stuck, just reload the page and you'll be able to make a fresh start.

- Also, each stage allows you to begin afresh, so you can just move on when you are ready.

- If you like, work together in pairs sharing a screen.

- The notebook has been designed to help support your learning by enabling you to explore, but also to challenge. However, we are not trying to trick you. In most cases, the answers are in front of you.

- If you have questions, please ask us!

## Stage 1 - Compute LCOE

According to [OEE](http://www.open-electricity-economics.org/book/text/03.html) the formula to calculate LCOE is as follows:

\\(LCOE = \frac{C_{fix} + \sum_{y=1}^Y CRF_y \cdot C_y}{\sum_{y=1}^Y CRF_y \cdot G_y}\\)

where LCOE is the levelized cost in EUR per kWh, \\(C_{fix}\\) is the capital investment costs incurred for setting up the project, \\(C_y\\) are operational costs incurred in year \\(y\\), \\(Y\\) is the technical lifetime in years, and \\(G_y\\) is electricity generation in kWh. The costs are called levelized because they are “leveled” over all units of output. Levelized costs can be calculated for a specific power plant or for generic types of generation technologies.

Capital recovery factor (CRF) is calculated for each year \\(y\\) of the plant's technical lifetime

\\(CRF_y =  (1 + r)^{-y}\\)

where r is the discount rate

Here we can store the technical lifetime of our technology in a |*variable* called ``technical_lifetime``

In [113]:
technical_lifetime = 20

You can access the value stored in the *variable* at anytime, by typing its name and press `shift` + `enter`

In [89]:
technical_lifetime

20

We then create an *array* of years using the value stored in ``technical_lifetime``. 

An *array* is like a list of values. The command ``np.arange()`` takes a number as an argument and creates a *array* of the same length containing the values from 0 to one less than the number.

In [90]:
year = np.arange(technical_lifetime)
year

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19])

Now we calculate CRF for each year in our plant's technical lifetime according to the formula we saw earlier:
\\(CRF_y =  (1 + r)^{-y}\\). Remember \\(r\\) is the discount rate, and \\(y\\) is the year.

`**` means "to the power of". 

**Q. What happens if you change the discount rate?**

In [112]:
discount_rate = 0.05
crf = (1 + discount_rate) ** - year
print(crf)

[1.         0.95238095 0.90702948 0.8638376  0.82270247 0.78352617
 0.7462154  0.71068133 0.67683936 0.64460892 0.61391325 0.58467929
 0.55683742 0.53032135 0.50506795 0.4810171  0.45811152 0.43629669
 0.41552065 0.39573396]


Now we've calculated CRF, let's get some data for the other parameters:

In [114]:
capital_costs = 1000.0  # €
operational_costs = np.repeat(500, technical_lifetime)  # €
electricity_generation = np.repeat(8760., technical_lifetime)  # kWh

The code above creates variables, each containing an *array* of values. We automatically generate the arrays using `np.repeat(50, technical_lifetime)`. This creates an array of length `technical_lifetime` where each element in the array contains the value `50`.

---

Remember the formula? Let's check it again:

\\(LCOE = \frac{C_{fix} + \sum_{y=1}^Y CRF_y \cdot C_y}{\sum_{y=1}^Y CRF_y \cdot G_y}\\)

Now, we can calculate LCOE:

In [115]:
lcoe = (capital_costs + sum(crf * operational_costs)) / sum(crf * electricity_generation)
print(lcoe)  # €/kWh

0.06580154242125368


At the moment, this seems a pretty arbitrary result. We're just putting in nonsense data, and we are getting nonsense in return. In stage 2, we'll start working with real data, but first, we need to make this calculation re-useable.

---

In Python, if we want to reuse a piece of code, we'll create a "function". To do this, you use the ``def`` command like this:

```python
def lcoe(capital_costs, operational_costs, electricity_generation, discount_rate, technical_lifetime):
    # <<Something should go here>>
    return value
```

- In the above example `lcoe` is the name of the function
- The list of names in the round brackets `()` are arguments
- Don't forget the `:` at the end of the list of arguments
- Use four spaces to indent your code
- The argument passed to `return` in the last line is what the function will give back when you run it

**Q. What should go in the function?**

Have a go below:

In [118]:
def lcoe(capital_costs, operational_costs, electricity_generation, discount_rate, technical_lifetime):

    # <<Write your code here>>
    value = "Replace me with some code"
    return value

Test your `lcoe` function by running the cell below.

In [119]:
actual = lcoe(5000, np.array([100, 100, 100, 100]), np.array([8760, 8760, 8760, 8760]), 0.1, 4)
assert actual == 0.1751093834091417, "That's not right, try again! The function returns '{}'".format(actual)

AssertionError: That's not right, try again! The function returns 'Replace me with some code'

### Stage 1 - Summary

If you managed to get it to work, well done! You've written a Python function which computes LCOE. If not, don't worry, the answer is below!

- In this stage, we introduced the LCOE equation
\\(LCOE = \frac{C_{fix} + \sum_{y=1}^Y CRF_y \cdot C_y}{\sum_{y=1}^Y CRF_y \cdot G_y}\\)
and implemented it using a Python function.
- We also explored how the capital recovery factors weighs future years differently as a function of the discount rate.
- We learnt a lot of Python concepts including *variables*, *arrays*, and *arguments*.
- We learnt how to reuse Python code by writing a function.

In the next stage, we'll extend our function to make it more useful and investigate the LCOE calculation itself.

### Stage 1 - Answer LCOE Function

As you can see, we just need to fill in the gaps. First build the array of years, calculate the array of CRF and then add the lcoe calculation.
```python
def lcoe(capital_costs, operational_costs, electricity_generation, discount_rate, technical_lifetime):
    year = np.arange(technical_lifetime)
    crf = (1 + discount_rate) ** - year
    value = (capital_costs + sum(crf * operational_costs)) / sum(crf * electricity_generation)
    return value
```

## Stage 2 - Explore Parameter Values

We'll now make use of an LCOE function to explore the effect of different parameter values for one technology - a combined cycle gas turbine. In the following implementation of LCOE, we make the assumption that the yearly values for operational costs and electricity generation are the same in each year of the technology's lifetime.

Here are the parameters for a combined cycle gas turbine (CCGT):

Parameter | Unit |Value
---|---|---
Lifetime| |25
Discount rate | |			8.0%
Station size | MW |	750.0
Overnight cost | €/kW | 750.0
Fixed O&M cost | €/kW	| 3.0
Efficiency | | 0.5
Fuel price | €/kWh| 0.03
Emission Factor	| ton/kW-yr| 1.8
Load factor | | 0.75

Here's the information we need to calculate the LCOE using the simple LCOE equation we developed earlier.

Parameter | Unit |Value
---|---|---
Capital cost | m€ | ?
Annual fixed O&M cost |	m€	| ?
Variable O&M cost |€/kWh| ?
Total fuel cost | €/yr | ?
Emissions Penalty |	€/ton	| ?

**Q. What's the levelised cost of electricity for the CCGT plant?**
- work through the list of inputs methodically. 
- Pay close attention to the units.  
- Use Python variables to store information you reuse.

In [153]:
# Add your working HERE

# 1. Populate some variables using the information from the table

lifetime = 25
discount_rate = 0.08
capacity = 750.0 # MW
overnight_cost = 750.0 # €/kW
fixed_om_cost = 3 # €/kW
efficiency = 0.5
fuel_price = 0.03 # €/kWh
load_factor = 0.75

# 2. Calculate the intermediate values (you can use Python like a calculator)
# Use *+-/ for multiply, add, substract and divide. Here's one to get you started...
# Remove the hashes on the following lines to make you calculations

capital_costs = capacity * overnight_cost
# electricity_generation = 
# operational_costs = 

# 3. Now use the earlier LCOE equation we developed to calculate LCOE
actual = lcoe(capital_costs, operational_costs, electricity_generation, discount_rate, lifetime)

assert actual == 0.06000000157662146, "Close, but no bananas. You got '{}'".format(actual)

AssertionError: Close, but no bananas. You got 'Replace me with some code'

### Hints

- Remember there are approximately `365 * 24 = 8760` hours in a year.
- The load factor describes the proportion of those 8760 hours the plant is operational.
- Operational costs are made up of fixed costs (not a function of activity) and variable costs (function of activity)
- The variable cost will be the product of the amount of fuel used and the fuel price.
- Fuel use can be computed from electricity generation and efficiency.

---
Now we extend our calculation of LCOE with a number of new parameters. `load_factor` represents the proportion of the hours in the year in which the technology generates electricity. We now calculate `annual_operational_cost` from two other new parameters `fuel_efficiency` and `fuel_cost` and `fixed_om_cost`.

In [154]:
def extended_lcoe(station_size, overnight_cost, fuel_efficiency, fuel_price, fixed_om_cost, load_factor, 
                discount_rate, technical_lifetime):
    """Calculates levelised cost of electricity as a function of useful parameters
    
    Arguments
    ---------
    station_size : float
        The capacity of the technology in kW
    overnight_cost : float
        The capital cost of the technology in €
    fuel_efficiency : float
        The ratio describing quantity of fuel required per unit of activity
    fuel_price : float
        The price paid per unit of input fuel in €/kWh
    fixed_om_cost : float
        The fixed operation and maintenance cost of the technology in €/kW
    load_factor : float
        The percentage of the year in which the technology generates electricity in %.
    discount_rate : float
        A decimal value less than 1
    technical_lifetime : int
        Technical lifetime of the technology in years
        
    Returns
    -------
    float
        The levelised cost of electricity in €/kWh
    """
    HOURS_IN_YEAR = 8760
    
    capital_cost = station_size * overnight_cost
    total_fixed_om_cost = station_size * fixed_om_cost
    
    annual_electricity_generation = station_size * HOURS_IN_YEAR * load_factor
    total_variable_om_cost = (annual_electricity_generation / fuel_efficiency) * fuel_price
    annual_operational_cost = total_fixed_om_cost + total_variable_om_cost
    
    year = np.arange(technical_lifetime)
    total_crf = sum((1 + discount_rate) ** - year)
    
    
    value = ((total_crf * annual_operational_cost) + capital_cost) / (total_crf * annual_electricity_generation)
    return value

So to perform the calculation again using our new LCOE function, we get:

In [155]:
ccgt_lcoe = extended_lcoe(capacity, overnight_cost, efficiency, fuel_price, fixed_om_cost, capacity * 8760 * 0.75, 
                          discount_rate, lifetime)
ccgt_lcoe

0.06000000157662146

### Q. What are the key parameters that determine the LCOE of the CCGT plant?

Run the next cell to create an interactive slider widget which allows you to play with the `extended_lcoe` function and answer the following questions:

1. Which is the most influential input parameter?
2. Are all the responses linear? Is it easy to tell?
3. What happens if gas is expensive?
4. If the price of the gas turbine doubles from €750/kW to €1500/kW, what's the effect upon LCOE at different load factors?
5. How important is discount rate compared to the fuel price?

In [156]:
interact(extended_lcoe, station_size=fixed(750000), overnight_cost=(500, 1500), fuel_efficiency=(0.3, 0.7), 
         fuel_price=(0.01, 0.10, 0.01), fixed_om_cost=3, load_factor=(0.01, 1.0, 0.1), 
         discount_rate=(0.01, 0.30, 0.01), technical_lifetime=25)

interactive(children=(IntSlider(value=1000, description='overnight_cost', max=1500, min=500), FloatSlider(valu…

<function __main__.extended_lcoe(station_size, overnight_cost, fuel_efficiency, fuel_price, fixed_om_cost, load_factor, discount_rate, technical_lifetime)>

### Stage 2 - Summary

- In this stage, we calculated the levelised cost for a CCGT plant.
- Then, we explored the LCOE equation interactively to try to understand what parameters influence the LCOE of a technology.

In the next stage, we compare LCOE across technologies.

# Stage 3: Comparing technologies

In the previous stages, we developed our understanding of the LCOE equation, and applied it to one technology. Now comes the *really* fun part, where we develop a comparison across different technologies.

We can now make a more advanced function which holds the logic to calculate LCOE when we have a number of extra input parameters. Now, we can pass a *list* (elements separated by commas inside square brackets) of parameter values, and the function returns a list of results.

In [164]:
def lcoe_params(station_sizes, overnight_costs, fixed_om_costs, load_factors, efficiencies, 
                 fuel_prices, discount_rates, technical_lifetimes, technology):
    """Calculate LCOE for cartesian product of all parameter lists
    """
    results = []  # Create a list to hold the observations
    
    # Loop over cartesian product of parameter values
    for (capacity, capex, fixed_om_cost, load_factor, efficiency, 
        fuel_price, discount_rate, technical_lifetime) in itertools.product(
        station_sizes, overnight_costs, fixed_om_costs, load_factors, 
        efficiencies, fuel_prices, discount_rates, technical_lifetimes):

        observation = {}  # Create a dictionary to store the observation
        
        lcoe = extended_lcoe(capacity, capex, efficiency, fuel_price, fixed_om_cost, 
                             load_factor, discount_rate, technical_lifetime)
    
        observation['Technology'] = technology
        observation['StationSize'] = capacity
        observation['OvernightCost'] = capex
        observation['FixedOMCost'] = fixed_om_cost
        observation['LoadFactor'] = load_factor
        observation['Efficiency'] = efficiency
        observation['FuelPrice'] = fuel_price
        observation['DiscountRate'] = discount_rate
        observation['Lifetime'] = technical_lifetime
        observation['LCOE'] = lcoe

        results.append(observation)
        
    return results

First, we'll use our new function to compare multiple technologies under their central operating characteristics, and explore how that changes under different discount rates.

In [175]:
# Here we create an array of load factors from 0.1 to 1.0 in steps of 0.01
load_factors = np.arange(0.1, 1.0, 0.01)
# We set the discount rate to a fixed value for the moment
discount_rate = [0.08] # %

# CCGT plant parameters
technical_lifetime = [25] # years
station_size = [750000]  # kW
fuel_price = [0.03] # €/kWh
overnight_cost = [750]  # €/kW
fixed_om_cost = [3.0] # €/kW
fuel_efficiency = [0.5]

ccgt_results = lcoe_params(
    station_size, overnight_cost, fixed_om_cost, load_factors, fuel_efficiency, 
    fuel_price, discount_rate, technical_lifetime, technology='CCGT')

# Coal plants
technical_lifetime = [35] # years
station_size = [1000000] # kW
fuel_price = [0.01] # €/kWh
overnight_cost = [2000]  # €/kW
fixed_om_cost = [5.0] # €/kW
fuel_efficiency = [0.4]

coal_results = lcoe_params(
    station_size, overnight_cost, fixed_om_cost, load_factors, fuel_efficiency, 
    fuel_price, discount_rate, technical_lifetime, technology='Coal')

# Nuclear plants
technical_lifetime = [50] # years
station_size = [1200000] # kW
fuel_price = [0.005] # €/kWh
overnight_cost = [4500]  # €/kW
fixed_om_cost = [2.0] # €/kW
fuel_efficiency = [0.4]

nuclear_results = lcoe_params(
    station_size, overnight_cost, fixed_om_cost, load_factors, fuel_efficiency, 
    fuel_price, discount_rate, technical_lifetime, technology='Nuclear')

# Concentrating solar power
technical_lifetime = [25] # years
station_size = [500000] # kW
fuel_price = [0.0] # €/kWh
overnight_cost = [3785]  # €/kW
fixed_om_cost = [5.0] # €/kW
fuel_efficiency = [1.0]

csp_results = lcoe_params(
    station_size, overnight_cost, fixed_om_cost, load_factors, fuel_efficiency, 
    fuel_price, discount_rate, technical_lifetime, technology='CSP')

# Wind turbines
technical_lifetime = [25] # years
station_size = [200000] # kW
fuel_price = [0.0] # €/kWh
overnight_cost = [1200]  # €/kW
fixed_om_cost = [2.0] # €/kW
fuel_efficiency = [1.0]

wind_results = lcoe_params(
    station_size, overnight_cost, fixed_om_cost, load_factors, fuel_efficiency, 
    fuel_price, discount_rate, technical_lifetime, technology='Wind')


plotting = pd.DataFrame(ccgt_results + coal_results + nuclear_results + csp_results + wind_results)
ex.line(plotting,x='LoadFactor', y='LCOE', color='Technology', facet_col='DiscountRate')

What do you see when you compare technologies at different load factors?